# 00 IMPORTS

Import all relevant libraries for later use.

In [2]:
from flask import Flask, render_template, request, jsonify
from werkzeug.utils import secure_filename
import os
from openai import OpenAI
import base64
import requests
from time import sleep

# 01 FUNCTIONS

Define all helper functions for later use.

In [3]:
# FUNCTION TO OPEN FILES
def open_file(filepath):
    with open(filepath, 'r', encoding='utf-8') as infile:
        return infile.read()
    
# Funktion, um Files zu speichern
def save_file(filepath, content):
    with open(filepath, 'w', encoding='utf-8') as outfile:
        outfile.write(content)
    
# FUNCTION TO DECODE FILES
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

# 02 FLASK APP & CREDENTIALS

Define app, credentials and relevant files for API calls.

In [4]:
# FLASK APP
app = Flask(__name__)
app.config['UPLOAD_FOLDER'] = 'uploads/'
app.config['MAX_CONTENT_LENGTH'] = 16 * 1024 * 1024

# CREDENTIALS
client = OpenAI(api_key=open_file('keys/openaiapikey.txt'))
system = open_file('prompts/system.txt')
user = open_file('prompts/user.txt')
model="gpt-4-turbo-2024-04-09"
api_key = open_file('keys/openaiapikey.txt')

# 03 API CALLS

Define functions to call OpenAI API (text and vision).

In [5]:
# FUNCTION TO CALL GPT CHATMODEL
def chatbot(messages, model="gpt-4-turbo-preview"):
    max_retry = 7
    retry = 0
    while True:
        try:
            response = client.chat.completions.create(model=model, messages=messages, temperature=0, max_tokens=1000, top_p=1, frequency_penalty=0, presence_penalty=0)
            text = response.choices[0].message.content
            return text
        except Exception as oops:
            print(f'\n\nError communicating with OpenAI: "{oops}"')
            if 'maximum context length' in str(oops):
                print('\n\n DEBUG: Trimming oldest message')
                continue
            retry += 1
            if retry >= max_retry:
                print(f"\n\nExiting due to excessive errors in API: {oops}")
                exit(1)
            print(f'\n\nRetrying in {2 ** (retry - 1) * 5} seconds...')
            sleep(2 ** (retry - 1) * 5)

# FUNCTION TO GPT-4 VISION
def gpt4_vision(filepath, user_input):
    
    # DECODE IMAGE
    base64_image = encode_image(filepath)
    user_request = f"Das ist der Inhalt meines Kühlschrankes. {user_input}. Was fehlt mir dazu noch?"
    print(user_request)
    
    # GPT4 VISION
    headers = {"Content-Type": "application/json", "Authorization": f"Bearer {api_key}"}
    payload = {"model": model, "messages": [{"role": "user", "content": [{"type": "text", "text": user_request},
                                                                          {"type": "image_url","image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}
                                                                           }]}],
               "max_tokens": 500, "temperature": 0
    }
    
    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

    # PARSE JSON RESPONSE
    response_json = response.json()
    message_content = response_json['choices'][0]['message']['content']
    
    return message_content

# 04 FLASK ROUTES

Define FLASK routes for Template, Text Input and Image Input.

In [6]:
@app.route("/")
def index():

    return render_template("index.html")

@app.route('/get', methods=['POST'])
def get_bot_response():
    
    # USER INPUT EINLESEN
    user_input = request.json['message']
    
    # PROMT ERSTELLEN
    message = [{'role': 'system', 'content': system}, {'role': 'user', 'content': user_input}]
    gpt_response = chatbot(message)
    
    # USER_INPUT SPEICHERN
    save_file('prompts/user_input.txt', user_input)
    
    # RESPONSE ZURÜCKGEBEN
    return jsonify(message=gpt_response)

@app.route('/upload', methods=['POST'])
def upload_file():
    
    user_input = open_file('prompts/user_input.txt')
    
    if 'file' not in request.files:
        return jsonify(message="No file part")
    file = request.files['file']
    if file.filename == '':
        return jsonify(message="No selected file")
    
    if file:
        filename = secure_filename(file.filename)
        filepath = os.path.join(app.config['UPLOAD_FOLDER'], filename)
        file.save(filepath)
        
        # Call GPT-4 Vision and add header
        recipe = gpt4_vision(filepath, user_input)
        print(recipe)
        
        return jsonify(message=recipe)
    
    return jsonify(message="Something went wrong")

# 05 START THE APP

Run the application on port 8082 (just a random port).

In [6]:
if __name__ == '__main__':
    port = 8082
    app.run(port=port)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8082
Press CTRL+C to quit
127.0.0.1 - - [23/Apr/2024 14:40:10] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/Apr/2024 14:40:10] "GET /static/css/style_static_V4.css HTTP/1.1" 304 -
127.0.0.1 - - [23/Apr/2024 14:40:10] "GET /static/js/chat_V4.js HTTP/1.1" 304 -
127.0.0.1 - - [23/Apr/2024 14:40:10] "GET /static/images/cook-icon.png HTTP/1.1" 200 -
127.0.0.1 - - [23/Apr/2024 14:40:10] "GET /static/fonts/Korb-Bold.woff2 HTTP/1.1" 304 -
127.0.0.1 - - [23/Apr/2024 14:40:22] "GET /static/images/user-icon.png HTTP/1.1" 200 -
127.0.0.1 - - [23/Apr/2024 14:40:22] "GET /static/fonts/SourceSansPro-Bold.woff2 HTTP/1.1" 200 -
127.0.0.1 - - [23/Apr/2024 14:40:22] "GET /static/fonts/SourceSansPro-Regular.woff2 HTTP/1.1" 200 -
127.0.0.1 - - [23/Apr/2024 14:40:25] "POST /get HTTP/1.1" 200 -


Das ist der Inhalt meines Kühlschrankes. I will einen Truthahn machen.. Was fehlt mir dazu noch?


127.0.0.1 - - [23/Apr/2024 14:40:52] "POST /upload HTTP/1.1" 200 -


Um einen Truthahn zuzubereiten, hast du bereits den Truthahn selbst im Kühlschrank. Für eine klassische Zubereitung könnten dir jedoch noch einige Zutaten fehlen, abhängig davon, wie du den Truthahn zubereiten möchtest. Hier sind einige Vorschläge:

1. **Gewürze und Kräuter**: Salz und Pfeffer sind Grundlagen, aber du könntest auch frische Kräuter wie Rosmarin, Thymian und Salbei hinzufügen, um den Geschmack zu verstärken. Diese sehe ich nicht in deinem Kühlschrank.

2. **Butter oder Öl**: Zum Einreiben des Truthahns, um eine knusprige Haut zu bekommen. Butter sehe ich in deinem Kühlschrank, das sollte ausreichen.

3. **Gemüse für die Füllung oder als Beilage**: Zwiebeln, Knoblauch, Möhren und Sellerie sind beliebt sowohl in der Füllung als auch als Aromageber beim Braten. Ich sehe einige Möhren und möglicherweise Knoblauch, aber Zwiebeln und Sellerie fehlen.

4. **Brühe oder Fond**: Zum Aufgießen während des Bratens, um den Truthahn saftig zu halten und später eine Sauce oder Gravy zu